# Venue and State table pipeline

Steps:
1. Load data from data folder
2. Extract table specific information into a pandas dataframe
3. Generate insert statements
4. Load data in SQL table

In [1]:
import os
os.chdir("../")

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from api.api_connect import fetch_api_data
import psycopg2 
from helpers import load_config, generate_insert_statements, write_insert_statements_to_file, run_script_file, load_data_file

config = load_config()

In [3]:
events = load_data_file("event_large")

In [4]:
# Initialize an empty list to store venue data
venue_list = []

# Initialize empty lists to keep track of existing venue_ids and venue_names
existing_venue_ids = []
existing_venue_names = []

# Loop through each event
for event in events:
    # Extract the venues information from the event
    venues = event["_embedded"]["venues"]
        
    # Loop through each venue for the event
    for venue in venues:
        # Extract the ID and name of the venue
        venue_id = venue["id"]
        venue_name = venue["name"].replace("'","''")
        
        # Check if venue_id is not already in the list
        if venue_id not in existing_venue_ids:
            # Add venue_id to the list of existing venue_ids
            existing_venue_ids.append(venue_id)
            
            # Extract additional venue information
            venue_location = venue.get("location", {})
            venue_postal_code = venue.get("postalCode", "")
            venue_lat = venue_location.get("latitude", "")
            venue_long = venue_location.get("longitude", "")
            venue_address = venue.get("address", {}).get("line1","")
            
            # Extract city name from the nested city field
            venue_city = venue.get("city", {}).get("name", "")
                   
            # Append venue information to the list
            venue_list.append({
                "venue_id": venue_id,
                "venue_name": venue_name,
                "venue_city": venue_city,
                "venue_postal_code": venue_postal_code,
                "venue_lat": venue_lat,
                "venue_long": venue_long,
                "venue_address": venue_address,
            })


In [5]:
# Create a DataFrame from the venue dictionary
venue_df = pd.DataFrame(data = venue_list)

In [6]:
venue_df.head()

,venue_id,venue_name,venue_city,venue_postal_code,venue_lat,venue_long,venue_address
0,KovZpZA7AAEA,Madison Square Garden,New York,10001,40.7497062,-73.9916006,7th Ave & 32nd Street
1,KovZpZA1J67A,Nationals Park,Washington,20003,38.873005,-77.007412,"1500 South Capitol Street, SE"
2,KovZpZAJaFkA,Levi''s® Stadium,Santa Clara,95054,37.40367671,-121.97034311,4900 Marie P. DeBartolo Way
3,KovZpZAdEt6A,Rice-Eccles Stadium,Salt Lake City,84112,40.7596422,-111.850261,451 South 1400 East
4,KovZ917ACh0,SoFi Stadium,Inglewood,90301,33.950529,-118.343767,1001 S. Stadium Dr


In [7]:
# Initialize an empty list to store state data
state_list = []

# Initialize an empty set to keep track of existing states
existing_state_codes = set()

# Loop through each event
for event in events:
    # Extract the venues information from the event
    venues = event["_embedded"]["venues"]
# Loop through each venue for the event
    for venue in venues:
        # Extract the state information of the venue
        state_name = venue.get("state", {}).get("name", "")
        state_code = venue.get("state", {}).get("stateCode", "")
        country_name = venue.get("country", {}).get("name", "")
        country_code = venue.get("country", {}).get("countryCode", "")
        
        # Check if the state is not empty and not already in the set of existing states
        if state_code not in existing_state_codes:
            # Add the state to the set of existing states
            existing_state_codes.add(state_code)
            
            # Append state information to the list
            state_list.append({
                "state_name": state_name,
                "state_code": state_code,
                "country_name":country_name,
                "country_code": country_code
            })

In [8]:
# Convert the list of states into a DataFrame
state_df = pd.DataFrame(state_list)

In [9]:
state_df.head()

,state_name,state_code,country_name,country_code
0,New York,NY,United States Of America,US
1,District of Columbia,DC,United States Of America,US
2,California,CA,United States Of America,US
3,Utah,UT,United States Of America,US
4,Washington,WA,United States Of America,US


In [10]:
insert_statements = generate_insert_statements(dataframe=state_df, table_name="state")
write_insert_statements_to_file(insert_statements, script_name="insert_state")
run_script_file("insert_state")

Insert statements written to 'sql_scripts/insert_state.sql' successfully!
SQL script executed successfully!


In [11]:
insert_statements = generate_insert_statements(dataframe=venue_df, table_name="venue")
write_insert_statements_to_file(insert_statements, script_name="insert_venue")
run_script_file("insert_venue")

Insert statements written to 'sql_scripts/insert_venue.sql' successfully!
SQL script executed successfully!
